In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive') #, force_remount=True

In [ ]:
#import os
#os.chdir('gdrive/My Drive/Colab Notebooks')

# annot df

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('все сегментации.xlsx')

In [ ]:
df.head(10)

In [ ]:
df.columns.tolist()

In [ ]:
df = df.drop(['Полина и Кеша','comment - С.Ю.', 'согласие у всех'], axis = 1)

In [ ]:
df = df.dropna(how='all')

In [ ]:
df['кеша'] = df['кеша'].fillna('None')
df['полина'] = df['полина'].fillna('None')
df['даша'] = df['даша'].fillna('None')

In [ ]:
df['id текста'] = df['id текста'].astype('int')

In [ ]:
df = df.drop(df[df['id текста'] == 10].index)

In [ ]:
df.tail()

# 1 text example

In [ ]:
df_text1 = df[df['id текста'] == 1]

In [ ]:
df_text1

In [ ]:
df_text1.shape

In [ ]:
annots = ['полина','кеша','даша']

In [ ]:
for index, row in df_text1.iterrows():
  print(index)
  print(row[annots[0]])
  print(row[annots[1]])
  print(row[annots[2]])
  print("\n")

In [ ]:
total_agree = []
agree = disagree = 0
for index, row in df_text1.iterrows():
  if row[annots[0]] == row[annots[1]] and row[annots[0]] == row[annots[2]]:
    agree += 1
  else:
    disagree += 1

#print(agree, disagree)
#print(agree/(agree+disagree))
print(agree/(agree+disagree-1)) 

In [ ]:
import sklearn
from sklearn.metrics import cohen_kappa_score

print(cohen_kappa_score(df_text1[annots[0]], df_text1[annots[1]]))
print(cohen_kappa_score(df_text1[annots[0]], df_text1[annots[2]]))
print(cohen_kappa_score(df_text1[annots[1]], df_text1[annots[2]]))

# all texts eval

In [ ]:
annots = ['полина','кеша','даша']

In [ ]:
def annot_agree(diff = False):
  """
  Согласие аннотаторов
  Annotators agreement
  """
  agree = disagree = 0
  for index, row in df.iterrows():
    if row[annots[0]] == row[annots[1]] and row[annots[0]] == row[annots[2]]:
      agree += 1
    else:
      disagree += 1
      if diff == True:
        print(f'row_id = {index}, text_id = {row["id текста"]}')
        for i in annots:
          print(f'{i}: {row[i]}')
        print('\n')
      else:
        continue
  return agree/(agree+disagree-1)

In [ ]:
annot_agree(diff=True)

In [ ]:
import sklearn
from sklearn.metrics import cohen_kappa_score

print(cohen_kappa_score(df[annots[0]], df[annots[1]]))
print(cohen_kappa_score(df[annots[0]], df[annots[2]]))
print(cohen_kappa_score(df[annots[1]], df[annots[2]]))

# deeppavlov

Тут я просто устанавливаю deeppavlov и чекаю его работу

In [ ]:
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install syntax_ru_syntagrus_bert

In [ ]:
!python -m deeppavlov install squad_bert

In [ ]:
!pip install tensorflow==1.14 

In [ ]:
!pip install russian_tagsets

In [ ]:
from deeppavlov import build_model, configs
model = build_model("ru_syntagrus_joint_parsing", download=True)

In [ ]:
text_parsed = []

sentences = ["Я шёл домой по незнакомой улице.", "Девушка пела в церковном хоре."]
for parse in model(sentences):
  sent_info = []
  row = parse.split('\t_\n')
  for i in row:
    inf = i.split('\t')
    sent_info.append(inf)
  text_parsed.append(sent_info)
  #print(parse, end="\n\n")

In [ ]:
mas = ['001','003','004','005']

In [ ]:
raw_texts = []
for i in mas:
  name = i+'_rus_raw.txt'
  text = open(name, 'r', encoding = 'utf-8').read()
  print(text)
  raw_texts.append(text)

In [ ]:
pip install rusenttokenize

In [ ]:
from rusenttokenize import ru_sent_tokenize

In [ ]:
raw_sents = []
for i in raw_texts:
  raw_sents.append(ru_sent_tokenize(i))

In [ ]:
for i in raw_sents:
  print(i)

In [ ]:
for parse in model(raw_sents[0]):
    print(parse, end="\n\n")

# text tok

In [ ]:
for index, row in df.iterrows():
  ann0 = row[annots[0]]
  ann1 = row[annots[1]]
  ann2 = row[annots[2]]
  edus = [ann0, ann1, ann2]
  for parse in model(edus):
    print(parse, end="\n\n")

In [ ]:
from deeppavlov import build_model, configs
model = build_model("ru_syntagrus_joint_parsing", download=True)

In [ ]:
text_parsed = []

sentences = ["Я шёл домой по незнакомой улице.", "Девушка пела в церковном хоре."]
for parse in model(sentences):
  sent_info = []
  row = parse.split('\t_\n')
  for i in row:
    inf = i.split('\t')
    sent_info.append(inf)
  text_parsed.append(sent_info)
  #print(parse, end="\n\n")

In [ ]:
annots_parsed = {}#{'полина':[],'кеша':[]}
for index, row in df.iterrows():
  text_parsed = []
  ann0 = row[annots[0]]
  ann1 = row[annots[1]]
  ann2 = row[annots[2]]
  edus = [ann0, ann1, ann2]
  for parse in model(edus):
     sent_info = []
     row = parse.split('\t_\n')
     for i in row:
       inf = i.split('\t')
       sent_info.append(inf)
     text_parsed.append(sent_info)
  for name_id in range(len(annots)):
    if annots[name_id] not in annots_parsed.keys():
      annots_parsed[annots[name_id]] = []
      annots_parsed[annots[name_id]].append(text_parsed[name_id])
    else:
      annots_parsed[annots[name_id]].append(text_parsed[name_id])

In [ ]:
for i in annots_parsed.keys():
  print(i)
  print(len(annots_parsed[i]))

In [ ]:
df.head()

In [ ]:
for name in annots:
  new_col_name = name+'_conllu'
  df[new_col_name] = annots_parsed[name]

In [ ]:
df.head()

In [ ]:
edus_norm = {}
for i in annots_parsed.keys():

  for row in annots_parsed[i]:
    tokens = []
    for j in row:
      tokens.append(j[2])
    row_tok = ' '.join(tokens)
    if i not in edus_norm.keys():
      edus_norm[i] = []
      edus_norm[i].append(row_tok)
    else:
      edus_norm[i].append(row_tok)

In [ ]:
for i in edus_norm.keys():
  print(i)
  print(len(edus_norm[i]))
  print(edus_norm[i][0])

In [ ]:
for name in annots:
  new_col_name = name+'_norm'
  df[new_col_name] = edus_norm[name]

In [ ]:
df.head()

In [ ]:
total_agree = []
agree = disagree = 0
for index, row in df.iterrows():
  if row[annots[0]+'_norm'] == row[annots[1]+'_norm'] and row[annots[0]+'_norm'] == row[annots[2]+'_norm']:
    agree += 1
  else:
    disagree += 1

#print(agree, disagree)
#print(agree/(agree+disagree))
print(agree/(agree+disagree-1)) 

In [ ]:
import sklearn
from sklearn.metrics import cohen_kappa_score

print(cohen_kappa_score(df[annots[0]+'_norm'], df[annots[1]+'_norm']))
print(cohen_kappa_score(df[annots[0]+'_norm'], df[annots[2]+'_norm']))
print(cohen_kappa_score(df[annots[1]+'_norm'], df[annots[2]+'_norm']))